In [28]:
from google.colab import files
uploaded = files.upload()


Saving flight_delays_with_iata_filled.csv to flight_delays_with_iata_filled.csv


In [30]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load your dataset
df = pd.read_csv("flight_delays_with_iata_filled.csv")

# Step 1: Clean airport name for fuzzy matching
df["airport_name_clean"] = df["Airport Name"].str.split(":").str[-1].str.lower().str.strip()

# Step 2: Create or reuse the IATA Code column
if "Airport Code" in df.columns:
    df["IATA Code"] = df["Airport Code"]  # use existing values
else:
    df["IATA Code"] = None  # create new column to fill in

# Step 3: Load OpenFlights airport reference data
url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"
cols = [
    "airport_id", "name", "city", "country", "iata_code", "icao_code",
    "latitude", "longitude", "altitude", "timezone", "dst", "tz", "type", "source"
]
airports = pd.read_csv(url, header=None, names=cols)

# Step 4: Clean reference airport names
airports = airports[airports["iata_code"].notna() & (airports["iata_code"] != "\\N")]
airports["name_clean"] = airports["name"].str.lower().str.strip()

# Step 5: Create list of reference names for fuzzy matching
airport_names_list = airports["name_clean"].tolist()

# Step 6: Define fuzzy matching function
def get_fuzzy_iata(name):
    match = process.extractOne(name, airport_names_list, scorer=fuzz.token_sort_ratio)
    if match and match[1] >= 85:
        return airports.loc[airports["name_clean"] == match[0], "iata_code"].values[0]
    return None

# Step 7: Apply fuzzy match to only missing IATA codes
df.loc[df["IATA Code"].isna(), "IATA Code"] = df.loc[df["IATA Code"].isna(), "airport_name_clean"].apply(get_fuzzy_iata)

# Step 8: Clean up helper column
df.drop(columns=["airport_name_clean"], inplace=True)

# Step 9: Save updated file
df.to_csv("flight_delays_iata_final.csv", index=False)
print("✅ Done! Saved as 'flight_delays_iata_final.csv'")

# Step 10: Print match summary
total = len(df)
matched = df["IATA Code"].notna().sum()
missing = df["IATA Code"].isna().sum()

print(f"Total rows: {total}")
print(f"Matched IATA codes: {matched}")
print(f"Still missing IATA codes: {missing}")


✅ Done! Saved as 'flight_delays_iata_final.csv'
Total rows: 376
Matched IATA codes: 164
Still missing IATA codes: 212


In [31]:
# Display the airport name along with the IATA code (including the unmatched ones)
airport_names_and_codes = df[["Airport Name", "IATA Code"]]

# Display the first few rows to check
print(airport_names_and_codes.head())

# Optionally, save this to a CSV file to inspect later
airport_names_and_codes.to_csv("airport_names_and_codes.csv", index=False)

print("✅ Saved as 'airport_names_and_codes.csv'")


                                        Airport Name IATA Code
0      Fayetteville, AR: Northwest Arkansas National      None
1            Las Vegas, NV: Harry Reid International      None
2         Kansas City, MO: Kansas City International       MCI
3  Brownsville, TX: Brownsville South Padre Islan...       BRO
4       Jacksonville, FL: Jacksonville International       JAX
✅ Saved as 'airport_names_and_codes.csv'


In [33]:
def get_fuzzy_iata(name):
    match = process.extractOne(name, airport_names_list, scorer=fuzz.token_sort_ratio)
    if match and match[1] >= 80:  # Lowered threshold to 80
        return airports.loc[airports["name_clean"] == match[0], "iata_code"].values[0]
    return None


In [34]:
# Display more rows with missing IATA codes
print(missing_rows[["Airport Name", "IATA Code"]].head(20))  # Adjust to show more rows if needed


                                         Airport Name IATA Code
0       Fayetteville, AR: Northwest Arkansas National      None
1             Las Vegas, NV: Harry Reid International      None
5             Detroit, MI: Detroit Metro Wayne County      None
6     Greer, SC: Greenville-Spartanburg International      None
7                      Tulsa, OK: Tulsa International      None
8    Savannah, GA: Savannah/Hilton Head International      None
9           Seattle, WA: Seattle/Tacoma International      None
10  Minneapolis, MN: Minneapolis-St Paul Internati...      None
11                   Juneau, AK: Juneau International      None
18                            New York, NY: LaGuardia      None
19                     Tampa, FL: Tampa International      None
20  Houston, TX: George Bush Intercontinental/Houston      None
21                 El Paso, TX: El Paso International      None
22       Cedar Rapids/Iowa City, IA: The Eastern Iowa      None
23                   Albany, NY: Albany 

In [37]:
print(df.columns)


Index(['Year', 'Month', 'Carrier', 'Arr Flights', 'Arr Del15', 'Carrier Ct',
       'Weather Ct', 'Nas Ct', 'Security Ct', 'Late Aircraft Ct',
       'Arr Cancelled', 'Arr Diverted', 'Carrier Delay', 'Weather Delay',
       'Nas Delay', 'Security Delay', 'Late Aircraft Delay', 'Total Delay',
       'Cancellation Rate', 'Actual Delay', 'Predicted Delay', 'Airport Name',
       'IATA Code'],
      dtype='object')


In [59]:
import pandas as pd

# Load your dataset
df = pd.read_csv("flight_delays_with_iata_filled.csv")

# Dictionary of airport names and their corresponding IATA codes
manual_iata_codes = {
    "Fayetteville, AR: Northwest Arkansas National": "XNA",
    "Las Vegas, NV: Harry Reid International": "LAS",
    "Detroit, MI: Detroit Metro Wayne County": "DTW",
    "Greer, SC: Greenville-Spartanburg International": "GSP",
    "Tulsa, OK: Tulsa International": "TUL",
    "Savannah, GA: Savannah/Hilton Head International": "SAV",
    "Seattle, WA: Seattle/Tacoma International": "SEA",
    "Minneapolis, MN: Minneapolis-St Paul International": "MSP",
    "Juneau, AK: Juneau International": "JNU",
    "New York, NY: LaGuardia": "LGA",
    "Tampa, FL: Tampa International": "TPA",
    "Houston, TX: George Bush Intercontinental/Houston": "IAH",
    "El Paso, TX: El Paso International": "ELP",
    "Cedar Rapids/Iowa City, IA: The Eastern Iowa": "CID",
    "Albany, NY: Albany International": "ALB",
    "Charleston/Dunbar, WV: West Virginia International Yeager": "CRW",
    "Miami, FL: Miami International": "MIA",
    "Akron, OH: Akron-Canton Regional": "CAK",
    "Rochester, NY: Frederick Douglass Greater Rochester International": "ROC",
    "Evansville, IN: Evansville Regional": "EVV",
    "Memphis, TN: Memphis International": "MEM",
    "Missoula, MT: Missoula Montana": "MSO",
    "Duluth, MN: Duluth International": "DLH",
    "Hays, KS: Hays Regional": "HYS",
    "Monroe, LA: Monroe Regional": "MLU",
    "Columbia, SC: Columbia Metropolitan": "CAE",
    "Montrose/Delta, CO: Montrose Regional": "MTJ",
    "International Falls, MN: Falls International Einarson Field": "INL",
    "Montgomery, AL: Montgomery Regional": "MGM",
    "La Crosse, WI: La Crosse Regional": "LSE",
    "Cincinnati, OH: Cincinnati/Northern Kentucky International": "CVG",
    "Atlanta, GA: Hartsfield-Jackson Atlanta International": "ATL",
    "Oakland, CA: Metro Oakland International": "OAK",
    "Moline, IL: Quad Cities International": "MLI",
    "Victoria, TX: Victoria Regional": "VCT",
    "Hartford, CT: Bradley International": "BDL",
    "Charleston, SC: Charleston AFB/International": "CHS",
    "Sioux Falls, SD: Joe Foss Field": "FSD",
    "Provo, UT: Provo Municipal": "PVU",
    "Oklahoma City, OK: Will Rogers World": "OKC",
    "Valparaiso, FL: Eglin AFB Destin Fort Walton Beach": "VPS",
    "Gunnison, CO: Gunnison-Crested Butte Regional": "GUC",
    "Sarasota/Bradenton, FL: Sarasota/Bradenton International": "SRQ",
    "Bangor, ME: Bangor International": "BGR",
    "Phoenix, AZ: Phoenix - Mesa Gateway": "AZA",
    "Wichita Falls, TX: Sheppard AFB/Wichita Falls Municipal": "SPS",
    "Fort Lauderdale, FL: Fort Lauderdale-Hollywood International": "FLL",
    "Little Rock, AR: Bill and Hillary Clinton Nat Adams Field": "LIT",
    "Richmond, VA: Richmond International": "RIC",
    "Greenville, NC: Pitt Greenville": "PGV",
    "Orlando, FL: Orlando International": "MCO",
    "South Bend, IN: South Bend International": "SBN",
    "Houston, TX: William P Hobby": "HOU",
    "Norfolk, VA: Norfolk International": "ORF",
    "Detroit, MI: Detroit Metro Wayne County": "DTW",
    "Charlotte Amalie, VI: Cyril E King": "STT",
    "Boston, MA: Logan International": "BOS",
    "Pensacola, FL: Pensacola International": "PNS",
    "Mason City, IA: Mason City Municipal": "MCW",
    "Harlingen/San Benito, TX: Valley International": "HRL",
    "Kearney, NE: Kearney Regional": "EAR",
    "Bangor, ME: Bangor International": "BGR",
    "Louisville, KY: Louisville Muhammad Ali International": "SDF",
    "Raleigh/Durham, NC: Raleigh-Durham International": "RDU",
    "Rochester, NY: Frederick Douglass Greater Rochester International": "ROC",
    "South Bend, IN: South Bend International": "SBN",
    "Las Vegas, NV: Harry Reid International": "LAS",
    "Houston, TX: William P Hobby": "HOU",
    "Appleton, WI: Appleton International": "ATW",
    "Richmond, VA: Richmond International": "RIC",
    "Tucson, AZ: Tucson International": "TUS",
    "Cedar Rapids/Iowa City, IA: The Eastern Iowa": "CID",
    "Baton Rouge, LA: Baton Rouge Metropolitan/Ryan Field": "BTR",
    "Roanoke, VA: Roanoke Blacksburg Regional": "ROA",
    "Montrose/Delta, CO: Montrose Regional": "MTJ",
    "Rochester, NY: Frederick Douglass Greater Rochester International": "ROC",
    "Sarasota/Bradenton, FL: Sarasota/Bradenton International": "SRQ",
    "El Paso, TX: El Paso International": "ELP",
    "Hayden, CO: Yampa Valley": "HDN",
    "Key West, FL: Key West International": "EYW",
    "Knoxville, TN: McGhee Tyson": "TYS",
    "El Paso, TX: El Paso International": "ELP",
    "Greer, SC: Greenville-Spartanburg International": "GSP",
    "Miami, FL: Miami International": "MIA",
    "Lake Charles, LA: Lake Charles Regional": "LCH",
    "Memphis, TN: Memphis International": "MEM",
    "Charlotte Amalie, VI: Cyril E King": "STT",
    "Seattle, WA: Seattle/Tacoma International": "SEA",
    "Charleston, SC: Charleston AFB/International": "CHS",
    "Minneapolis, MN: Minneapolis-St Paul International": "MSP",
    "Houston, TX: William P Hobby": "HOU",
    "Stillwater, OK: Stillwater Regional": "SWO",
    "Cincinnati, OH: Cincinnati/Northern Kentucky International": "CVG",
    "Hartford, CT: Bradley International": "BDL",
    "Spokane, WA: Spokane International": "GEG",
    "Roanoke, VA: Roanoke Blacksburg Regional": "ROA",
    "Lansing, MI: Capital Region International": "LAN",
    "Baton Rouge, LA: Baton Rouge Metropolitan/Ryan Field": "BTR",
    "Minneapolis, MN: Minneapolis-St Paul International": "MSP",
    "Burbank, CA: Bob Hope": "BUR",
    "Reno, NV: Reno/Tahoe International": "RNO",
    "Bozeman, MT: Bozeman Yellowstone International": "BZN",
    "Mission/McAllen/Edinburg, TX: McAllen International": "MFE",
    "Sioux Falls, SD: Joe Foss Field": "FSD",
    "Rochester, NY: Frederick Douglass Greater Rochester International": "ROC",
    "Denver, CO: Denver International": "DEN",
    "Norfolk, VA: Norfolk International": "ORF",
    "Lansing, MI: Capital Region International": "LAN",
    "Columbia, MO: Columbia Regional": "COU",
    "Provo, UT: Provo Municipal": "PVU",
    "Shreveport, LA: Shreveport Regional": "SHV",
    "St. Petersburg, FL: St Pete Clearwater International": "PIE",
    "Lansing, MI: Capital Region International": "LAN",
    "Traverse City, MI: Cherry Capital": "TVC",
    "Memphis, TN: Memphis International": "MEM",
    "South Bend, IN: South Bend International": "SBN",
    "Valparaiso, FL: Eglin AFB Destin Fort Walton Beach": "VPS",
    "Detroit, MI: Detroit Metro Wayne County": "DTW",
    "Roanoke, VA: Roanoke Blacksburg Regional": "ROA",
    "Seattle, WA: Seattle/Tacoma International": "SEA",
    "Helena, MT: Helena Regional": "HLN",
    "Columbia, SC: Columbia Metropolitan": "CAE",
    "Houston, TX: William P Hobby": "HOU",
    "Ontario, CA: Ontario International": "ONT",
    "Knoxville, TN: McGhee Tyson": "TYS",
    "Fayetteville, AR: Northwest Arkansas National": "XNA",
    "Providence, RI: Rhode Island TF Green International": "PVD",
    "Louisville, KY: Louisville Muhammad Ali International": "SDF",
    "Minot, ND: Minot International": "MOT",
    "Melbourne, FL: Melbourne Orlando International": "MLB",
    "Iron Mountain/Kingsfd, MI: Ford": "IMT",
    "Bangor, ME: Bangor International": "BGR",
    "Lake Charles, LA: Lake Charles Regional": "LCH",
    "Richmond, VA: Richmond International": "RIC",
    "Belleville, IL: Scott AFB MidAmerica St Louis": "BLV",
    "Las Vegas, NV: Harry Reid International": "LAS",
    "Kotzebue, AK: Ralph Wien Memorial": "OTZ",
    "Portland, OR: Portland International": "PDX",
    "Seattle, WA: Seattle/Tacoma International": "SEA",
    "Boise, ID: Boise Air Terminal": "BOI",
    "Ontario, CA: Ontario International": "ONT",
    "Burbank, CA: Bob Hope": "BUR",
    "Moline, IL: Quad Cities International": "MLI",
    "Portland, OR: Portland International": "PDX",
    "Tampa, FL: Tampa International": "TPA",
    "Asheville, NC: Asheville Regional": "AVL",
    "Fort Lauderdale, FL: Fort Lauderdale-Hollywood International": "FLL",
    "Harlingen/San Benito, TX: Valley International": "HRL",
    "Jackson, WY: Jackson Hole": "JAC",
    "Sioux Falls, SD: Joe Foss Field": "FSD",
    "Tampa, FL: Tampa International": "TPA",
    "Detroit, MI: Detroit Metro Wayne County": "DTW",
    "White Plains, NY: Westchester County": "HPN",
    "Houston, TX: George Bush Intercontinental/Houston": "IAH",
    "Burbank, CA: Bob Hope": "BUR",
    "Greer, SC: Greenville-Spartanburg International": "GSP",
    "Atlanta, GA: Hartsfield-Jackson Atlanta International": "ATL",
    "Raleigh/Durham, NC: Raleigh-Durham International": "RDU",
    "Charleston, SC: Charleston AFB/International": "CHS",
    "Spokane, WA: Spokane International": "GEG",
    "Oklahoma City, OK: Will Rogers World": "OKC",
    "Fort Lauderdale, FL: Fort Lauderdale-Hollywood International": "FLL",
    "Oklahoma City, OK: Will Rogers World": "OKC",
    "Memphis, TN: Memphis International": "MEM",
    "Rockford, IL: Chicago/Rockford International": "RFD",
    "Tulsa, OK: Tulsa International": "TUL",
    "Tucson, AZ: Tucson International": "TUS",
    "Albany, NY: Albany International": "ALB",
    "Raleigh/Durham, NC: Raleigh-Durham International": "RDU",
    "Richmond, VA: Richmond International": "RIC",
    "Christiansted, VI: Henry E. Rohlsen": "STX",
    "Savannah, GA: Savannah/Hilton Head International": "SAV",
    "Providence, RI: Rhode Island TF Green International": "PVD",
    "Valparaiso, FL: Eglin AFB Destin Fort Walton Beach": "VPS",
    "Atlanta, GA: Hartsfield-Jackson Atlanta International": "ATL",
    "Tucson, AZ: Tucson International": "TUS",
    "Cedar Rapids/Iowa City, IA: The Eastern Iowa": "CID",
    "Baton Rouge, LA: Baton Rouge Metropolitan/Ryan Field": "BTR",
    "Roanoke, VA: Roanoke Blacksburg Regional": "ROA",
    "Montrose/Delta, CO: Montrose Regional": "MTJ",
    "Rochester, NY: Frederick Douglass Greater Rochester International": "ROC",
    "Sarasota/Bradenton, FL: Sarasota/Bradenton International": "SRQ",
    "El Paso, TX: El Paso International": "ELP",
    "Hayden, CO: Yampa Valley": "HDN",
    "Key West, FL: Key West International": "EYW",
    "Knoxville, TN: McGhee Tyson": "TYS",
    "El Paso, TX: El Paso International": "ELP",
    "Greer, SC: Greenville-Spartanburg International": "GSP",
    "Miami, FL: Miami International": "MIA",
    "Lake Charles, LA: Lake Charles Regional": "LCH",
    "Memphis, TN: Memphis International": "MEM",
    "Charlotte Amalie, VI: Cyril E King": "STT",
    "Seattle, WA: Seattle/Tacoma International": "SEA",
    "Charleston, SC: Charleston AFB/International": "CHS",
    "Minneapolis, MN: Minneapolis-St Paul International": "MSP",
    "Houston, TX: William P Hobby": "HOU",
    "Stillwater, OK: Stillwater Regional": "SWO",
    "Cincinnati, OH: Cincinnati/Northern Kentucky International": "CVG",
    "Hartford, CT: Bradley International": "BDL",
    "Spokane, WA: Spokane International": "GEG",
    "Roanoke, VA: Roanoke Blacksburg Regional": "ROA",
    "Lansing, MI: Capital Region International": "LAN",
    "Baton Rouge, LA: Baton Rouge Metropolitan/Ryan Field": "BTR",
    "Minneapolis, MN: Minneapolis-St Paul International": "MSP",
    "Burbank, CA: Bob Hope": "BUR",
    "Reno, NV: Reno/Tahoe International": "RNO",
    "Bozeman, MT: Bozeman Yellowstone International": "BZN",
    "Mission/McAllen/Edinburg, TX: McAllen International": "MFE",
    "Sioux Falls, SD: Joe Foss Field": "FSD",
    "Rochester, NY: Frederick Douglass Greater Rochester International": "ROC",
    "Denver, CO: Denver International": "DEN",
    "Norfolk, VA: Norfolk International": "ORF",
    "Lansing, MI: Capital Region International": "LAN",
    "Columbia, MO: Columbia Regional": "COU",
    "Provo, UT: Provo Municipal": "PVU",
    "Shreveport, LA: Shreveport Regional": "SHV",
    "St. Petersburg, FL: St Pete Clearwater International": "PIE",
    "Lansing, MI: Capital Region International": "LAN",
    "Traverse City, MI: Cherry Capital": "TVC",
    "Memphis, TN: Memphis International": "MEM",
    "South Bend, IN: South Bend International": "SBN",
    "Valparaiso, FL: Eglin AFB Destin Fort Walton Beach": "VPS",
    "Detroit, MI: Detroit Metro Wayne County": "DTW",
    "Roanoke, VA: Roanoke Blacksburg Regional": "ROA",
    "Seattle, WA: Seattle/Tacoma International": "SEA",
    "Helena, MT: Helena Regional": "HLN",
    "Columbia, SC: Columbia Metropolitan": "CAE",
    "Houston, TX: William P Hobby": "HOU",
    "Ontario, CA: Ontario International": "ONT",
    "Knoxville, TN: McGhee Tyson": "TYS",
    "Fayetteville, AR: Northwest Arkansas National": "XNA",
    "Providence, RI: Rhode Island TF Green International": "PVD",
    "Louisville, KY: Louisville Muhammad Ali International": "SDF",
    "Minot, ND: Minot International": "MOT",
    "Melbourne, FL: Melbourne Orlando International": "MLB",
    "Iron Mountain/Kingsfd, MI: Ford": "IMT",
    "Bangor, ME: Bangor International": "BGR",
    "Lake Charles, LA: Lake Charles Regional": "LCH",
    "Richmond, VA: Richmond International": "RIC",
    "Belleville, IL: Scott AFB MidAmerica St Louis": "BLV",
    "Las Vegas, NV: Harry Reid International": "LAS",
    "Kotzebue, AK: Ralph Wien Memorial": "OTZ",
    "Portland, OR: Portland International": "PDX",
    "Seattle, WA: Seattle/Tacoma International": "SEA",
    "Boise, ID: Boise Air Terminal": "BOI",
    "Ontario, CA: Ontario International": "ONT",
    "Burbank, CA: Bob Hope": "BUR",
    "Moline, IL: Quad Cities International": "MLI",
    "Portland, OR: Portland International": "PDX",
    "Tampa, FL: Tampa International": "TPA",
    "Asheville, NC: Asheville Regional": "AVL",
    "Fort Lauderdale, FL: Fort Lauderdale-Hollywood International": "FLL",
    "Harlingen/San Benito, TX: Valley International": "HRL",
    "Jackson, WY: Jackson Hole": "JAC",
    "Sioux Falls, SD: Joe Foss Field": "FSD",
    "Tampa, FL: Tampa International": "TPA",
    "Detroit, MI: Detroit Metro Wayne County": "DTW",
    "White Plains, NY: Westchester County": "HPN",
    "Houston, TX: George Bush Intercontinental/Houston": "IAH",
    "Burbank, CA: Bob Hope": "BUR",
    "Greer, SC: Greenville-Spartanburg International": "GSP",
    "Atlanta, GA: Hartsfield-Jackson Atlanta International": "ATL",
    "Raleigh/Durham, NC: Raleigh-Durham International": "RDU",
    "Charleston, SC: Charleston AFB/International": "CHS",
    "Spokane, WA: Spokane International": "GEG",
    "Oklahoma City, OK: Will Rogers World": "OKC",
    "Fort Lauderdale, FL: Fort Lauderdale-Hollywood International": "FLL",
    "Oklahoma City, OK: Will Rogers World": "OKC",
    "Memphis, TN: Memphis International": "MEM",
    "Rockford, IL: Chicago/Rockford International": "RFD",
    "Tulsa, OK: Tulsa International": "TUL",
    "Tucson, AZ: Tucson International": "TUS",
    "Albany, NY: Albany International": "ALB",
    "Raleigh/Durham, NC: Raleigh-Durham International": "RDU",
    "Richmond, VA: Richmond International": "RIC",
    "Christiansted, VI: Henry E. Rohlsen": "STX",
    "Savannah, GA: Savannah/Hilton Head International": "SAV",
    "Providence, RI: Rhode Island TF Green International": "PVD",
    "Valparaiso, FL: Eglin AFB Destin Fort Walton Beach": "VPS",
    "Atlanta, GA: Hartsfield-Jackson Atlanta International": "ATL",
    "Missoula, MT: Missoula Montana": "MSO",
    "Norfolk, VA: Norfolk International": "ORF",
    "Asheville, NC: Asheville Regional": "AVL",
    "Seattle, WA: Seattle/Tacoma International": "SEA",
    "Lafayette, LA: Lafayette Regional Paul Fournet Field": "LFT",
    "Detroit, MI: Detroit Metro Wayne County": "DTW",
    "Bristol/Johnson City/Kingsport, TN: Tri Cities": "TRI",
    "Monterey, CA: Monterey Regional": "MRY",
    "Oakland, CA: Metro Oakland International": "OAK",
    "Midland/Odessa, TX: Midland International Air and Space Port": "MAF",
    "Fargo, ND: Hector International": "FAR",
    "Cedar Rapids/Iowa City, IA: The Eastern Iowa": "CID",
    "Boise, ID: Boise Air Terminal": "BOI",
    "Hayden, CO: Yampa Valley": "HDN",
    "Tallahassee, FL: Tallahassee International": "TLH",
    "Asheville, NC: Asheville Regional": "AVL",
    "Bozeman, MT: Bozeman Yellowstone International": "BZN",
    "Midland/Odessa, TX: Midland International Air and Space Port": "MAF",
    "Las Vegas, NV: Harry Reid International": "LAS",
    "Shreveport, LA: Shreveport Regional": "SHV",
    "Fargo, ND: Hector International": "FAR",
    "Columbia, SC: Columbia Metropolitan": "CAE",
    "Mobile, AL: Mobile Regional": "MOB",
    "Cincinnati, OH: Cincinnati/Northern Kentucky International": "CVG",
    "Joplin, MO: Joplin Regional": "JLN",
    "Grand Junction, CO: Grand Junction Regional": "GJT",
    "Buffalo, NY: Buffalo Niagara International": "BUF",
    "Rochester, NY: Frederick Douglass Grtr Rochester International": "ROC",
    "San Juan, PR: Luis Munoz Marin International": "SJU",
    "Milwaukee, WI: General Mitchell International": "MKE",
    "Jackson/Vicksburg, MS: Jackson Medgar Wiley Evers International": "JAN",
    "Amarillo, TX: Rick Husband Amarillo International": "AMA",
    "Santa Barbara, CA: Santa Barbara Municipal": "SBA",
    "Albuquerque, NM: Albuquerque International Sunport": "ABQ",
    "Indianapolis, IN: Indianapolis International": "IND",
    "Portland, ME: Portland International Jetport": "PWM",
    "Madison, WI: Dane County Regional-Truax Field": "MSN",
    "Syracuse, NY: Syracuse Hancock International": "SYR",
    "Eau Claire, WI: Chippewa Valley Regional": "EAU",
    "Honolulu, HI: Daniel K Inouye International": "HNL",
    "Corpus Christi, TX: Corpus Christi International": "CRP",
    "Pittsburgh, PA: Pittsburgh International": "PIT",
    "Liberal, KS: Liberal Mid-America Regional": "LBL",
    "Birmingham, AL: Birmingham-Shuttlesworth International": "BHM",
    "Cleveland, OH: Cleveland-Hopkins International": "CLE",
    "Omaha, NE: Eppley Airfield": "OMA",
    "Colorado Springs, CO: City of Colorado Springs Municipal": "COS",
    "Bend/Redmond, OR: Roberts Field": "RDM",
    "Kona, HI: Ellison Onizuka Kona International at Keahole": "KOA",
    "West Palm Beach/Palm Beach, FL: Palm Beach International": "PBI",
    "Augusta, GA: Augusta Regional at Bush Field": "AGS",
    "San Antonio, TX: San Antonio International": "SAT",
    "San Luis Obispo, CA: San Luis County Regional": "SBP",
    "Durango, CO: Durango La Plata County": "DRO",
    "Myrtle Beach, SC: Myrtle Beach International": "MYR",
    "Ashland, WV: Tri-State/Milton J. Ferguson Field": "HTS",
    "Palm Springs, CA: Palm Springs International": "PSP",
    "Fort Myers, FL: Southwest Florida International": "RSW",
    "Bloomington/Normal, IL: Central Il Regional Airport at Bloomington": "BMI",
    "Austin, TX: Austin - Bergstrom International": "AUS",
    "San Jose, CA: Norman Y. Mineta San Jose International": "SJC",
    "Huntsville, AL: Huntsville International-Carl T Jones Field": "HSV",
    "Dillingham, AK: Dillingham Airport": "DLG",
    "Newark, NJ: Newark Liberty International": "EWR",
    "Dayton, OH: James M Cox/Dayton International": "DAY",
    "Columbus, OH: Rickenbacker International": "LCK",
    "Harrisburg, PA: Harrisburg International": "MDT",
    "Long Beach, CA: Long Beach Airport": "LGB",
    "Santa Ana, CA: John Wayne Airport-Orange County": "SNA",
    "Meridian, MS: Key Field": "MEI",
    "Sanford, FL: Orlando Sanford International": "SFB",
    "Des Moines, IA: Des Moines International": "DSM",
    "St. Louis, MO: St Louis Lambert International": "STL",
    "Sacramento, CA: Sacramento International": "SMF",
    "Great Falls, MT: Great Falls International": "GTF",
    "Bishop, CA: Bishop Airport": "BIH",
    "Alexandria, LA: Alexandria International": "AEX",
    "Scranton/Wilkes-Barre, PA: Wilkes Barre Scranton International": "AVP",
    "Medford, OR: Rogue Valley International - Medford": "MFR",
    "Lubbock, TX: Lubbock Preston Smith International": "LBB",
    "Nashville, TN: Nashville International": "BNA",
    "Columbus, OH: John Glenn Columbus International": "CMH",
    "Grand Island, NE: Central Nebraska Regional": "GRI"

}
# Manually assign the IATA codes based on the list
for airport, iata in manual_iata_codes.items():
    df.loc[df['Airport Name'] == airport, 'IATA Code'] = iata

# Save the updated DataFrame to a new CSV
df.to_csv("flight_delays_with_iata_updated.csv", index=False)

print("IATA codes manually updated!")



IATA codes manually updated!


In [60]:
# Check how many rows still have missing IATA codes
missing_iata_count = df[df['IATA Code'].isna()].shape[0]
print(f"Number of rows with missing IATA codes: {missing_iata_count}")

# Optionally, view a few rows with missing IATA codes (if any)
if missing_iata_count > 0:
    print("\nRows with missing IATA codes:")
    print(df[df['IATA Code'].isna()].head())  # Display the first few rows with missing IATA codes
else:
    print("All IATA codes have been successfully updated!")


Number of rows with missing IATA codes: 0
All IATA codes have been successfully updated!


In [61]:
# Save the updated DataFrame to a new CSV
df.to_csv("flight_delays_with_iata_updated.csv", index=False)

print("CSV file with updated IATA codes saved!")


CSV file with updated IATA codes saved!


In [62]:
from google.colab import files
files.download('flight_delays_with_iata_updated.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>